<a href="https://colab.research.google.com/github/AlexandreGuirado/Dataset-SpamData/blob/main/Dataset_SpamData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Bibliotecas
import random
import spacy
import pandas as pd
import seaborn as sns
from spacy.util import minibatch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt


In [ ]:
# dados
data_path = "https://raw.githubusercontent.com/AlexandreGuirado/Dataset-SpamData/main/spam.csv"
data = pd.read_csv(data_path, encoding="ISO-8859-1")
print(data.head())

In [ ]:
data = pd.read_csv(data_path, encoding="ISO-8859-1")
observations = len(data.index)
print(f"Tamanho do Dataset: {observations}\n")
print(data['v1'].value_counts())
print()
print(data['v1'].value_counts() / len(data.index) * 100.0)

In [ ]:
# Criamos um modelo vazio com o idioma do dataset
nlp = spacy.blank("en")

# Criamos agora um classificador de texto com classes exclusivas + arquitetura "bow" 
text_cat = nlp.create_pipe(
              "textcat",
              config={
                "exclusive_classes": True,
                "architecture": "bow"})

# Adicionamos o classificador a nosso modelo vazio
nlp.add_pipe(text_cat)

# Adicionamos as "classes exclusivas"
text_cat.add_label("ham")
text_cat.add_label("spam")

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
      data['v2'], data['v1'], test_size=0.33, random_state=7)

In [ ]:
train_lables = [{'cats': {'ham': label == 'ham',
                          'spam': label == 'spam'}}  for label in y_train]

test_lables = [{'cats': {'ham': label == 'ham',
                      'spam': label == 'spam'}}  for label in y_test]

# Spacy model data
train_data = list(zip(x_train, train_lables))
test_data = list(zip(x_test, test_lables))

In [ ]:
def train_model(model, train_data, optimizer, batch_size, epochs=10):
    losses = {}
    random.seed(1)

    for epoch in range(epochs):
        random.shuffle(train_data)

        batches = minibatch(train_data, size=batch_size)
        for batch in batches:
            # Split batch into texts and labels
            texts, labels = zip(*batch)

            # Update model with texts and labels
            model.update(texts, labels, sgd=optimizer, losses=losses)
        print("Loss: {}".format(losses['textcat']))

    return losses['textcat']

In [ ]:
optimizer = nlp.begin_training()
batch_size = 5
epochs = 10

# Treinar!
train_model(nlp, train_data, optimizer, batch_size, epochs)

In [ ]:
print(train_data[0])
sample_test = nlp(train_data[0][0])
print(sample_test.cats)

In [ ]:
def get_predictions(model, texts):
    # Tokenizar
    docs = [model.tokenizer(text) for text in texts]

    # textcat para verificar os scores
    textcat = model.get_pipe('textcat')
    scores, _ = textcat.predict(docs)

    # Mostramos os scores mais altos com argmax
    predicted_labels = scores.argmax(axis=1)
    predicted_class = [textcat.labels[label] for label in predicted_labels]

    return predicted_class

In [ ]:
# ACURÁCIA
train_predictions = get_predictions(nlp, x_train)
test_predictions = get_predictions(nlp, x_test)

train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

print(f"Train accuracy: {train_accuracy}")
print(f"Test accuracy: {test_accuracy}")

In [ ]:
# MATRIZ DE CONFUSÃO TREINO
print("TREINO:")
cf_train_matrix = confusion_matrix(y_train, train_predictions)
plt.figure(figsize=(10,8))
sns.heatmap(cf_train_matrix, annot=True, fmt='d')

In [ ]:
# MATRIZ DE CONFUSÃO TESTE
print("TESTE:")
cf_test_matrix = confusion_matrix(y_test, test_predictions)
plt.figure(figsize=(10,8))
sns.heatmap(cf_test_matrix, annot=True, fmt='d')